# Releasing an assignment

**These instructions are currently incomplete**.

Creating the release version of an assignment consists of creating a
student version of a notebook that has solutions omited, based on an already created teacher, or master, version.

To create the master/teacher notebook that has exercises, solutions and tests, see the documentation on the ["Create Assignment" extension](2 - Developing assignments.ipynb). 

In the `release_example` directory we have an two example master/teacher notebooks:

* [Problem 1](release_example/teacher/Problem 1.ipynb)
* [Problem 2](release_example/teacher/Problem 2.ipynb)

There is also a [header notebook](release_example/header.ipynb) which will be prepended to the teacher notebooks in the release version.

In [ ]:
cd release_example

In [ ]:
!ls -R

Now, to create the release version, all we have to do is to run the following command. This command specifies that:

* we should include all the notebooks in the `teacher` directory
* we should copy them to the `student` directory
* the header file is `header.ipynb` (this file will be prepended to the beginning of the assignment files)

For other options, please see the [`nbgrader assign` documentation](../command_line_tools/nbgrader assign.ipynb).

In [ ]:
%%bash

nbgrader assign teacher/*.ipynb \
    --build-dir=student \
    --IncludeHeaderFooter.header=header.ipynb

These are the resulting student notebooks:

* [Problem 1](release_example/student/Problem 1.ipynb)
* [Problem 2](release_example/student/Problem 2.ipynb)

They are versions of the assignment that can be distributed to the students, who can fill out the relevant locations of the notebook with their solutions, and eventually turn it back in.

## Recording grade cells

If you want to record information about grade cells into the database (for example, to then use those grades and point values, rather than the ones in the notebooks students have returned), then you can optionally run `nbgrader assign` with the `--save-cells` flag.

Before we can run this command, though, we need to actually what the assignment is that we're recording grade cells for. We can do this using the API provided by nbgrader, which provides access to a database to store information about students and their grades:

In [ ]:
# remove an existing database
import os
dbpath = "/tmp/nbgrader_example.db"
if os.path.exists(dbpath):
    os.remove(dbpath)

# create a connection to the db using the nbgrader API
from nbgrader.api import Gradebook
gb = Gradebook("sqlite:///" + dbpath)

In [ ]:
# add the assignment to the database
gb.add_assignment("Problem Set 1")

Now, we can run our `nbgrader` command. Note that in addition to `--save-cells`, we need to specify the name of the assignment (which should be the same as the one in the database) and the name of the database (which should be the same as the one we just created):

In [ ]:
%%bash

nbgrader assign teacher/*.ipynb \
    --build-dir=student \
    --IncludeHeaderFooter.header=header.ipynb \
    --save-cells \
    --assignment="Problem Set 1" \
    --db="sqlite:////tmp/nbgrader_example.db"

After doing this, we should be able to see the grade cells in the database:

In [ ]:
# print out how many 
gb = Gradebook("sqlite:///" + dbpath)
notebook = gb.find_notebook("Problem 1", "Problem Set 1")
print("There are {} grade cells in Problem Set 1, Problem 1".format(len(notebook.grade_cells)))